# SIFT image alignment tutorial

SIFT (Scale-Invariant Feature Transform) is an algorithm developped by David Lowe in 1999. It is a worldwide reference for image alignment and object recognition. The robustness of this method enables to detect features at different scales, angles and illumination of a scene. Silx provides an implementation of SIFT in OpenCL, meaning that it can run on Graphics Processing Units and Central Processing Units as well. Interest points are detected in the image, then data structures called *descriptors* are built to be characteristic of the scene, so that two different images of the same scene have similar descriptors. They are robust to transformations like translation, rotation, rescaling and illumination change, which make SIFT interesting for image stitching. In the fist stage, descriptors are computed from the input images. Then, they are compared to determine the geometric transformation to apply in order to align the images. This implementation can run on most graphic cards and CPU, making it usable on many setups. OpenCL processes are handled from Python with PyOpenCL, a module to access OpenCL parallel computation API.

This tutuorial explains the three subsequent steps:

* keypoint extraction
* Keypoint matching
* image alignment

All the tutorial has been made using the Jupyter notebook.

In [ ]:
%pylab inline

In [ ]:
# display test image
import scipy.misc
image = scipy.misc.ascent()
imshow(image, cmap="gray")

In [ ]:
#Initialization of the sift object is time consuming: it compiles all the code.
import os 
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "0" #set to 1 to see the compilation going on
from silx.image import sift
%time sift_ocl = sift.SiftPlan(template=image, devicetype="CPU") #switch to GPU to test your graphics card

In [ ]:
print("Time for calculating the keypoints on one image of size %sx%s"%image.shape)
%time keypoints = sift_ocl(image)
print("Number of keypoints: %s"%len(keypoints))
print("Keypoint content:")
print(keypoints.dtype)
print("x: %.3f \t y: %.3f \t sigma: %.3f \t angle: %.3f" % 
      (keypoints[-1].x,keypoints[-1].y,keypoints[-1].scale,keypoints[-1].angle))
print("descriptor:")
print(keypoints[-1].desc)

In [ ]:
#Overlay keypoints on the image:
imshow(image, cmap="gray")
plot(keypoints[:].x, keypoints[:].y,".")

In [ ]:
#Diplaying keypoints by scale:
hist(keypoints[:].scale, 100)
xlabel("scale")

In [ ]:
#One can see 2 groups of keypoints: <12 and >12. Let's display them using colors.
S = 8
L = 20
tiny = keypoints[keypoints[:].scale<S]
small = keypoints[numpy.logical_and(keypoints[:].scale<L,keypoints[:].scale>=S)]
bigger = keypoints[keypoints[:].scale>=L]
imshow(image, cmap="gray")
plot(tiny[:].x, tiny[:].y,".g", label="tiny")
plot(small[:].x, small[:].y,".b", label="small")
plot(bigger[:].x, bigger[:].y,".r", label="large")
legend()

## Image matching and alignment

Matching can also be performed on the device (GPU) as every single keypoint from an image needs to be compared with all
keypoints from the second image.

In this simple example we will simple offset the first image by a few pixels

In [ ]:
shifted = numpy.zeros_like(image)
shifted[5:,8:] = image[:-5, :-8]
shifted_points = sift_ocl(shifted)

In [ ]:
%time mp = sift.MatchPlan()
%time match = mp.match(keypoints, shifted_points)
print("Number of Keypoints with for image 1 : %i, For image 2 : %i, Matching keypoints: %i" % (kp1.size, kp2.size, match.shape[0]))



### References

- David G. Lowe, Distinctive image features from scale-invariant keypoints, International Journal of Computer Vision, vol. 60, no 2, 2004, p. 91–110 - "http://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf"

